# LabChronicle — Maintenance ledger entry

This notebook creates a **maintenance event YAML** from the repo's fully documented template and stores it under `maintenance/events/<microscope>/<YYYY>/`.

**Rule:** humans describe work performed; no pass/fail logic here.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys
import os
from datetime import datetime, timezone

repo_root = Path.cwd().resolve()
if not (repo_root / "src").exists():
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root / "src"))

import yaml
import ipywidgets as W
from IPython.display import display, clear_output

from aic_chronicle.yaml_io import load_template, save_yaml
from aic_chronicle.ids import make_maintenance_id, slugify
from aic_chronicle import github_ops
from aic_chronicle import widgets as AW

TEMPLATE_PATH = repo_root / "templates" / "maintenance_template.yaml"
INSTRUMENTS_DIR = repo_root / "instruments"


## 1) Select microscope

In [ ]:
mic_dd, get_selection = AW.instrument_selector(INSTRUMENTS_DIR)
out = W.Output()

def _render(*_):
    with out:
        clear_output()
        sel = get_selection()
        if sel is None:
            print("No selection.")
            return
        print(f"Instrument: {sel.metadata.display_name}  [{sel.metadata.instrument_id}]")
        print(f"File: {sel.metadata.filepath.name}")

mic_dd.observe(_render, "value")
_render()
display(W.VBox([mic_dd, out]))


## 2) Create maintenance event
Fill in the form, click **Initialize**, then **Save**.


In [ ]:
service_provider = W.Dropdown(
    options=[("vendor","vendor"), ("distributor","distributor"), ("third_party","third_party"), ("internal","internal"), ("unknown","unknown")],
    value="vendor",
    description="service_provider:",
    layout=W.Layout(width="55%")
)
company = AW.text_widget("company", placeholder="provider company", value="", width="65%")
contact = AW.text_widget("contact", placeholder="name/email/phone", value="", width="65%")
reference = AW.text_widget("reference", placeholder="ticket/case/PO/invoice", value="", width="65%")

started_utc = AW.datetime_widget("started_utc")
ended_utc = AW.text_widget("ended_utc (optional)", placeholder="YYYY-MM-DDThh:mm:ssZ", value="", width="55%")

reason = W.Dropdown(
    options=[("scheduled","scheduled"), ("problem","problem"), ("upgrade","upgrade"), ("install","install"), ("check","check"), ("other","other")],
    value="scheduled",
    description="reason:",
    layout=W.Layout(width="55%")
)
reason_details = AW.notes_widget("reason_details")

action = W.Dropdown(
    options=[("service","service"), ("repair","repair"), ("align","align"), ("calibrate","calibrate"), ("clean","clean"), ("replace","replace"), ("update","update"), ("other","other")],
    value="service",
    description="action:",
    layout=W.Layout(width="55%")
)
action_details = AW.notes_widget("action_details")

microscope_status_after = W.Dropdown(
    options=[("in_service","in_service"), ("limited","limited"), ("out_of_service","out_of_service")],
    value="in_service",
    description="status_after:",
    layout=W.Layout(width="55%")
)
downtime_hours = W.FloatText(value=0.0, description="downtime_hours:", layout=W.Layout(width="55%"))

related_qc = AW.notes_widget("related_qc (one per line)")
tags = AW.text_widget("tags (comma-separated)", placeholder="optics, alignment, vendor", value="", width="75%")

attach_upload = AW.file_upload_widget("Upload attachment(s)", accept="", multiple=True)
attach_uri = AW.notes_widget("attachment URI(s) (one per line)")

followup = AW.notes_widget("followup")
next_due_date = AW.text_widget("next_due_date (optional)", placeholder="YYYY-MM-DD", value="", width="35%")

notes = AW.notes_widget("notes")

init_btn = W.Button(description="Initialize maintenance YAML", button_style="success")
save_btn = W.Button(description="Save maintenance YAML", button_style="success")

preview = W.Textarea(value="", description="YAML preview:", layout=W.Layout(width="95%", height="260px"))
preview.disabled = True
msg = W.Output()

state = {"maint": None, "maintenance_id": None, "attachments": []}  # attachments: list[(filename, bytes)]

def _init(_):
    with msg:
        clear_output()
        sel = get_selection()
        if sel is None:
            print("Select microscope first.")
            return
        m = load_template(TEMPLATE_PATH)
        m["microscope"] = sel.metadata.instrument_id
        m["started_utc"] = started_utc.value.strip() or AW.utc_now_iso()
        m["ended_utc"] = ended_utc.value.strip() if ended_utc.value.strip() else ""
        slug = slugify(reason.value + "_" + action.value)[:16]
        mid = make_maintenance_id(sel.metadata.instrument_id, m["started_utc"], slug)
        m["maintenance_id"] = mid

        m["service_provider"] = service_provider.value
        m["company"] = company.value
        m["contact"] = contact.value
        m["reference"] = reference.value

        m["reason"] = reason.value
        m["reason_details"] = reason_details.value
        m["action"] = action.value
        m["action_details"] = action_details.value

        m["microscope_status_after"] = microscope_status_after.value
        m["downtime_hours"] = float(downtime_hours.value) if downtime_hours.value else 0.0

        # linkages
        rq = [ln.strip() for ln in related_qc.value.splitlines() if ln.strip()]
        m["related_qc"] = rq

        # attachments saved at Save time
        m["attachments"] = []
        m["tags"] = [t.strip() for t in tags.value.split(",") if t.strip()]
        m["followup"] = followup.value
        m["next_due_date"] = next_due_date.value.strip()
        m["notes"] = notes.value

        # store uploads
        atts = []
        if attach_upload.value:
            for fn, meta in attach_upload.value.items():
                atts.append((fn, meta["content"]))
        state["attachments"] = atts

        # store URI attachments
        uri_lines = [ln.strip() for ln in attach_uri.value.splitlines() if ln.strip()]
        if uri_lines:
            m["attachments"].extend(uri_lines)

        state["maint"] = m
        state["maintenance_id"] = mid

        preview.value = yaml.safe_dump(m, sort_keys=False, allow_unicode=True, default_flow_style=False, width=120)
        print(f"Initialized maintenance_id: {mid}")

init_btn.on_click(_init)

def _save(_):
    with msg:
        clear_output()
        if state["maint"] is None:
            print("Initialize first.")
            return
        m = state["maint"]
        sel = get_selection()
        year = (m.get("started_utc") or AW.utc_now_iso())[:4]
        event_dir = repo_root / "maintenance" / "events" / sel.metadata.instrument_id / year
        att_dir = event_dir / "attachments" / m["maintenance_id"]
        event_dir.mkdir(parents=True, exist_ok=True)
        att_dir.mkdir(parents=True, exist_ok=True)

        # save uploaded attachments
        for fn, content in (state.get("attachments") or []):
            out_path = att_dir / fn
            out_path.write_bytes(content)
            rel = str(out_path.relative_to(repo_root)).replace(os.sep, "/")
            m["attachments"].append(rel)

        yaml_path = event_dir / f"{m['maintenance_id']}.yaml"
        save_yaml(yaml_path, m)
        preview.value = yaml.safe_dump(m, sort_keys=False, allow_unicode=True, default_flow_style=False, width=120)
        print(f"Saved maintenance YAML: {yaml_path.relative_to(repo_root)}")
        if state.get("attachments"):
            print(f"Saved {len(state['attachments'])} uploaded attachment(s) under: {att_dir.relative_to(repo_root)}")

save_btn.on_click(_save)

display(W.VBox([
    W.HTML("<h4>Provider</h4>"),
    service_provider, company, contact, reference,
    W.HTML("<h4>Timing</h4>"),
    W.HBox([started_utc, ended_utc]),
    W.HTML("<h4>Reason</h4>"),
    reason, reason_details,
    W.HTML("<h4>Action</h4>"),
    action, action_details,
    W.HTML("<h4>Impact</h4>"),
    microscope_status_after, downtime_hours,
    W.HTML("<h4>Linkages & attachments</h4>"),
    related_qc, tags,
    attach_upload, attach_uri,
    W.HTML("<h4>Follow-up</h4>"),
    followup, next_due_date,
    W.HTML("<h4>Notes</h4>"),
    notes,
    W.HBox([init_btn, save_btn]),
    preview,
    msg
]))


## 3) Optional GitHub operations
If this repo is a git clone (contains `.git`), you can commit/push the new maintenance record.


In [ ]:
git_panel = AW.git_commit_push_panel(
    repo_root,
    is_ready=lambda: state.get("maint") is not None,
    default_branch=lambda: (f"add-maint-{state['maint']['maintenance_id']}" if state.get("maint") else "add-maint"),
    default_commit_message=lambda: (f"Add maintenance event {state['maint']['maintenance_id']}" if state.get("maint") else "Add maintenance event"),
    header="Optional GitHub operations",
)

display(git_panel)
